In [60]:
import pandas as pd
import netpixi
from graph_tool import Graph
from netpixi.data.gt import *
from graph_tool import centrality
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("data.csv")

In [3]:
df_artista = df.artists

In [4]:
mask = (df['release_date'] > "1990-01-01") & (df['release_date'] <= "2020-05-15")

In [5]:
df_recorte = df.loc[mask]

In [6]:
artistas = df_recorte['artists'].tolist()

In [7]:
flat_list = []
lista_artistas_musicas = []
for sublist in artistas:
    sub = sublist.strip(" ] [ ").split(", ")
    lista_artistas_musicas.append(sub)
    for e in sub:
        flat_list.append(e.replace("'",""))

In [33]:
artistas_certo = []
sub_sub = []
for i in lista_artistas_musicas:
    sub_sub = []
    for e in i:
        e = e.replace("'","")
        sub_sub.append(e.strip('""'))
    artistas_certo.append(sub_sub)

In [34]:
df_recorte['artists'] = artistas_certo

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
total_artista = set(flat_list)


## criando dicionário com todos os artistas e sua popularidade média de músicas

In [36]:
pop = df_recorte["popularity"].tolist()

In [37]:
count = 0
pop_new = []
for a in artistas_certo:
    for g in a:
        pop_new.append(pop[count])
    count += 1

In [38]:
pop_musica_artista = pd.DataFrame({
    "artista":flat_list,
    "popularidade": pop_new
})

In [39]:
pop_musica_artista

,artista,popularidade
0,Paul Simon,35
1,Concrete Blonde,35
2,Roberta Flack,35
3,Donny Hathaway,35
4,Benny Hinn,37
...,...,...
75538,DripReport,75
75539,Tyga,75
75540,Cash Cash,70
75541,Andy Grammer,70


In [40]:
df_artista_media_pop = pop_musica_artista.groupby("artista").mean().reset_index()

In [41]:
df_artista_media_pop.artista = df_artista_media_pop.artista.str.strip('""')

In [42]:
df_artista_media_pop

,artista,popularidade
0,Adolescents Orquesta,60.111111
1,Alexander ONeal,45.333333
2,Amy Hanaialii,37.000000
3,Aulii Cravalho,66.800000
4,B. DAddario as Flounder,44.000000
...,...,...
15566,辛曉琪,46.500000
15567,那英,46.500000
15568,高慧君,52.000000
15569,黃品源,47.000000


In [43]:
musicas_com_feat = []
for e in artistas_certo:
    if len(e) > 1:
        musicas_com_feat.append(e)

In [44]:
musicas_com_feat

[['Roberta Flack', 'Donny Hathaway'],
 ['Ice Cube', 'Flavor Flav'],
 ['Wolfgang Amadeus Mozart',
  'Takako Nishizaki',
  'Capella Istropolitana',
  'Johannes Wildner'],
 ['Wolfgang Amadeus Mozart',
  'Takako Nishizaki',
  'Capella Istropolitana',
  'Johannes Wildner'],
 ['Dave Stewart', 'Candy Dulfer'],
 ['Yo-Yo', 'Ice Cube'],
 ['U2', 'The Edge', 'Johnny Cash'],
 ['Leigh Harline', 'Richard Hayman', 'Richard Hayman Symphony Orchestra'],
 ['Flaco Jimenez', 'Dwight Yoakam'],
 ['Beastie Boys', 'Biz Markie'],
 ['R. Kelly', 'Public Announcement'],
 ['Stan Getz', 'Astrud Gilberto'],
 ['David Lee Garza', 'Los Musicales'],
 ['Dr. Dre', 'Jewell'],
 ['Newsies Ensemble', 'Newsies Additional Singing Cast'],
 ['Fourplay', 'Chaka Khan', 'Nathan East'],
 ['Lee Ritenour', 'Harvey Mason', 'Bob James'],
 ['Helmet', 'House Of Pain'],
 ['Depeche Mode', 'Alan Wilder', 'Steve Lyon'],
 ['Happy Mondays', 'andy weatherall', 'Paul Oakenfold'],
 ['Ludwig van Beethoven', 'Mischa Maisky', 'Martha Argerich'],
 ['Jer

In [45]:
musicas_com_feat_flat = [item.strip('""') for sublist in musicas_com_feat for item in sublist]


In [46]:
artista_participantes_feat = list(set(musicas_com_feat_flat))

### Criando rede

In [47]:
g = Graph(directed=False)
g.vp['Nome'] = g.new_vp('object')
g.vp['Pop_media'] = g.new_vp('object')
g.ep["Numero_parcerias"] = g.new_ep("object")

### Criando nós

In [48]:
for art in range(0,len(artista_participantes_feat)):
    g.add_vertex()
    g.vp["Nome"][art] = artista_participantes_feat[art]
    g.vp["Pop_media"][art] = df_artista_media_pop.loc \
    [df_artista_media_pop["artista"] == artista_participantes_feat[art],"popularidade"].iloc[0]

In [49]:
dic = {}
for artista in artista_participantes_feat:
    dic[artista] = {}
    for musica in musicas_com_feat:
        if artista in musica:
            for participante in musica:
                if artista != participante:
                    #dic[artista].append(participante)
                    if participante not in dic[artista]:
                        dic[artista][participante] = 1
                    else:
                        dic[artista][participante] += 1
                 

In [50]:
dic

{'6LACK': {'Young Thug': 1,
  'Normani': 1,
  'Selena Gomez': 1,
  'Future': 1,
  'Phora': 1,
  'Summer Walker': 1,
  'Snoh Aalegra': 1,
  'J. Cole': 1,
  'Khalid': 2,
  'Ty Dolla $ign': 1,
  'Sabrina Claudio': 1,
  'T-Pain': 1,
  'Jhené Aiko': 1,
  'Vince Staples': 1,
  'Mereba': 1,
  'Jessie Reyez': 2,
  'Bea Miller': 1,
  'Kiana Ledé': 1},
 'Novo Amor': {'Ed Tullett': 1},
 'Nea': {'Felix Jaehn': 1},
 'Jamie Grace': {'TobyMac': 1},
 'D-Loc': {'Tech N9ne': 1, 'Big Krizz Kaliko': 1, 'Dalima': 1},
 'Yolanda Be Cool': {'DCup': 2, 'JT': 1},
 'Conkarah': {'Shaggy': 1, 'DJ Fle': 1},
 'Aminé': {'Nelly': 1, 'Gucci Mane': 1, 'Kehlani': 1},
 'Felly': {'MAX': 1, 'Technotronic': 1},
 'Spongebob': {'Sandy': 2, 'Mr. Krabs': 2, 'Plankton & Patrick': 2},
 'Juhn Allstar': {'Alex Rose': 1,
  'Casper Magico': 1,
  'Nengo Flow': 1,
  'Bryant Myers': 1,
  'Noriel': 1,
  'Miky Woodz': 1,
  'Jhay Cortez': 1,
  'Myke Towers': 1},
 'Sammy Johnson': {'Tree Vaifale': 1, 'Fiji': 1},
 'Göran Söllscher': {'Jean Si

In [51]:
contador = 0
for artista, participantes in dic.items():
    for participante, ocorrencia in participantes.items():
        no1 = artista_participantes_feat.index(artista)
        no2 = artista_participantes_feat.index(participante)
        g.add_edge(no1,no2)
        contador += 1
        
        
        
        
        
        
        
        

In [54]:
dc = degree_gt(g)
g.vp['degree'] = dc

bc, _ = centrality.betweenness(g)
g.vp['betweenness'] = bc

cc = centrality.closeness(g)
g.vp['closeness'] = cc

_, ec = centrality.eigenvector(g)
g.vp['eigenvector'] = ec

In [57]:
df_completo = pd.DataFrame({
    "artista":artista_participantes_feat
})

In [58]:
df_completo = pd.DataFrame({k: list(v) for k, v in g.vp.items()})

In [59]:
df_completo

,Nome,Pop_media,degree,betweenness,closeness,eigenvector
0,6LACK,65.357143,0.004495,0.002426,0.225495,3.945695e-04
1,Novo Amor,62.833333,0.000250,0.000000,1.000000,1.278256e-122
2,Nea,84.000000,0.000250,0.000000,0.122049,3.861627e-10
3,Jamie Grace,48.500000,0.000250,0.000000,0.148679,1.013342e-07
4,D-Loc,49.000000,0.000749,0.000000,0.191606,2.649360e-05
...,...,...,...,...,...,...
8005,Lil Yachty,65.242424,0.008740,0.001927,0.232972,7.939017e-04
8006,Lil Villian,35.000000,0.000999,0.000000,0.700000,3.831809e-74
8007,Flaco Jimenez,40.500000,0.000499,0.000038,0.137371,1.646194e-08
8008,Kristen Bell,69.166667,0.001748,0.000496,0.148904,1.407039e-07


In [63]:
import statsmodels.formula.api as smf

model = smf.ols(data=df_completo, formula='Pop_media ~ degree') # z1 é explicada por x1?
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Pop_media   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     35.37
Date:                Fri, 05 Nov 2021   Prob (F-statistic):           2.84e-09
Time:                        20:45:26   Log-Likelihood:                -30056.
No. Observations:                8010   AIC:                         6.012e+04
Df Residuals:                    8008   BIC:                         6.013e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     52.0633      0.133    391.262      0.000      51.802      52.324
degree       338.3197     56.884      5.948      0.000     226.813     449.826
==============================================================================
Omnibus:                      161.635   Durbin-Watson:                   2.039
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              158.972
Skew:                           0.316   Prob(JB):                     3.02e-35
Kurtosis:                       2.724   Cond. No.                         494.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""